# 把官方raw数据集的一部分转换回图片

pytorch 和 anaconda 的库中均有 libiomp5md.dll
用到这个 dll 时会导致冗余报错
一种方法是允许冗余
另一种是直接找到删一个

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]='True' # 允许冗余

In [2]:
from torchvision.datasets import mnist
from skimage import io

路径内无对应数据则会下载，保存在磁盘的是 raw
.data .targes 可以看源码，返回的是_load_data()
_load_data() 返回的就是参考代码的 read_image_file(),read_label_file()
这个返回的是 raw 划分后并且转 tensor，和 transformer 参数无关
其中，默认的是 transformer.toImage()，而非 transformer.toTensor()
transformer 在 DataLoader 时才会起作用，而且 PIL.Image 类型传给 dataloader 无法正常使用，只能是 tensor
toImage 后怎么读出不清楚，如果可以的话，那样就可以用可以用 PIL 来 save 图像了
就不必用 skimage 做 ndarray 保存为 jpg
其实还有其他许多展示方法，这是初学的我所最初了解的

In [3]:
train=mnist.MNIST('official_data',train=True)
test=mnist.MNIST('official_data',train=False)

In [4]:
X=train.data # old_usage: train.train_data
X

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [5]:
X.size()

torch.Size([60000, 28, 28])

In [6]:
y=train.targets # old_usage: train.train_labels
y

tensor([5, 0, 4,  ..., 5, 6, 8])

In [7]:
for i in range(10):
    io.imsave("images/"+str(i)+'.jpg',X[i].numpy())

参考代码：
import os
from skimage import io
import torchvision.datasets.mnist as mnist

root="D:/MNIST/data/MNIST/raw"
train_set=(
    mnist.read_image_file(os.path.join(root, 'train-images-idx3-ubyte')),
    mnist.read_label_file(os.path.join(root, 'train-labels-idx1-ubyte'))
)
test_set=(
    mnist.read_image_file(os.path.join(root, 't10k-images-idx3-ubyte')),
    mnist.read_label_file(os.path.join(root, 't10k-labels-idx1-ubyte'))
)
print("training set :",train_set[0].size())
print("test set :",test_set[0].size())

def convert_to_img(train=True):
    if(train):
        f=open(root+'train.txt','w')
        data_path=root+'/train/'
        if(not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(train_set[0],train_set[1])):
            img_path=data_path+str(i)+'.jpg'
            io.imsave(img_path,img.numpy())
            f.write(img_path+' '+str(label)+'\n')
        f.close()
    else:
        f = open(root + 'test.txt', 'w')
        data_path = root + '/test/'
        if (not os.path.exists(data_path)):
            os.makedirs(data_path)
        for i, (img,label) in enumerate(zip(test_set[0],test_set[1])):
            img_path = data_path+ str(i) + '.jpg'
            io.imsave(img_path, img.numpy())
            f.write(img_path + ' ' + str(label) + '\n')
        f.close()

convert_to_img(True)#转换训练集
convert_to_img(False)#转换测试集